# PRAKTIKUM Natural Language Processing

----------
## **Thema: Klassifikation von Dokumenten**

Durch die Lehrveranstaltungen Machine Learning 1 und Machine Learning 2 konnten Sie sich bereits einiges an Wissen im Bereich des Maschinellen Lernens aneignen. Machine Learning spielt auch beim Natural Language Processing eine wichtige Rolle. In der Vorlesung und im Praktikum werden wir uns daher damit beschäftigen, wie das bereits Gelernte auf natürlichsprachlichen Daten angewendet werden kann, aber natürlich auch welche zusätzlichen Methoden und Techniken erforderlich sind. 

Ziel dieser Praktikumseinheit ist es, dass Sie die grundlegende Vorgehensweise bei der Textklassifikation mit sklearn, die Sie bereits in ML1 gelernt haben, nochmals wiederholen und Ihr bereits vorhandenes Wissen so festigen. Um die zu verarbeitenden Textdaten in einen Feature-Vektor zu überführen, verwenden wir eine einfache, auch in sklearn verfügbare Transformationsmethode. Im Verlauf der Vorlesung und des Praktikums werden Sie noch andere (für manche Anwendungen effektivere) Methoden kennenlernen. 

### Lernziele: 
* Eine saubere Klassifikation durchführen können (mit besonderem Fokus auf der korrekten Verwendung von Trainings-, Validierungs- und Testdaten)
* Das sklearn-Pipelining-Konzept erklären und korrekt anwenden können
* Den sklearn-Transformator ``CountVectorizer`` anwenden können und erklären können, welche Art von Vektoren durch ihn erzeugt werden


### Lernkontrolle: 

Bei diesem Übungsblatt findet keine formale Lernkontrolle statt. Sie werden die erlernten (bzw. wiederholten) Methoden und Techniken allerdings in späteren Praktikumsstunden wieder benötigen.

Unten finden Sie allerdings noch Tipps zur eigenständigen Lernkontrolle. 

-----------
## **Lernmaterial**


1. Lehrvortrag zu Term-Dokument-Vektoren
2. Material zur Wiederholung des Pipelining-Konzepts von sklearn 
3. Vorschläge für konkrete Übungsaufgaben 
4. Tipps zur Lernkontrolle

### Pipelining-Konzept von sklearn

Das Konzept des Pipelinings wurde im letzten Semester bereits in ML1 eingeführt. Tipp: Schauen Sie sich hierzu das damals zur Verfügung gestellte Jupyter-Notebook mit Beispielen nochmals an (siehe dazu den unteren Abschnitt im Notebook zu Data Transformation) sowie den zur Erklärung verwendeten Foliensatz. 

Darüber hinaus gibt es viele Internet-Ressourcen, die das Konzept des Pipelinings erklären. 
Beispielsweise: 
* https://www.kaggle.com/baghern/a-deep-dive-into-sklearn-pipelines
* https://scikit-learn.org/stable/modules/compose.html

Tipp: Nicht jeder Person liegt dieselbe Art der Erklärung. Suchen Sie ggf. nach einem Dokument, das "Ihre Sprache" spricht. Ein weiterer Tipp: Ziehen Sie bei der Suche nach "Ihrer Lernressource" auch Lehrbücher in Betracht. Im Unterschied zu Internet-Ressourcen haben Lehrbücher den Vorteil, dass Sie auf mehr als nur eine Frage eine Antwort bieten. Sobald Sie sich einmal an den jeweiligen Stil gewöhnt haben (sich quasi eingelesen haben), finden Sie dort somit auf die nächsten Fragen eine schnelle, in Ihr jeweiliges Denkmodell gut passende Antwort! 


### Übungsaufgabe

Im Unterordner "data/bbc" finden Sie englischsprachige Nachrichtentexte. Diese sind in 5 Kategorien eingeteilt. Trainieren Sie einen Klassifikator, der in der Lage ist, ein ungelabeltes Dokument einer der 5 Kategorien zuzuordnen. (Vereinfachung: Sie können sich auch auf zwei beliebige Kategorien beschränken.) Verwenden Sie zur Transformation der Dokumente als Feature-Vektoren den ``CountVectorizer`` aus sklearn. Arbeiten Sie mit Trainings-, Validierungs- und Testdaten. Verwenden Sie außerdem eine sklearn ``Pipeline``. Verwenden Sie entweder NaiveBayes (``MultinomialNB``) oder eine SVM als Klassifikator. Bestimmen Sie zunächst mittels Kreuzvalidierung die aktuelle Accuracy Ihres Modells. Führen Sie anschließend eine GridSearch durch, um gute Parametereinstellungen zu finden. Evaluieren Sie zum Schluss das Modell auf den Testdaten. 

#### Tipps: 
* Machen Sie sich falls erforderlich zunächst nochmals mit dem Konzept des Pipelinings vertraut (beispielsweise indem Sie hierfür das zur Verfügung gestellte Material verwenden)
* Skizzieren Sie sich zunächst kurz die einzelnen Schritte und ihre Reihenfolge. 
* Zögern Sie nicht um Hilfe zu bitten, wenn Sie irgendwo hängen! 



### Tipps für eine eigenständige Lernkontrolle:

Möglichkeiten zur eigenen Lernkontrolle nach der Bearbeitung der Lerneinheit: 
* Stellen Sie sich vor, Sie müssten anschließend für strukturierte Daten einen Klassifikator trainieren. Was würde sich im Fall von strukturierten Daten in Ihrem Code ändern? 
* Erklären Sie das Konzept des Pipelinings einer anderen Person! (Oder machen Sie sich schriftlich Notizen dazu. Tipp: Nur zu überlegen, ob man es verstanden hat, ist nicht dasselbe wie es zu erklären oder zu notieren!)
* Bei der Einführung des Pipelining-Konzepts in ML1 wurde auch der ``ColumnTransformer`` vorgestellt. Wann wird ein ColumnTransformer benötigt und wie wird er beim Pipelining verwendet?
* Erklären Sie, wieso zusätzlich zu den Validierungsdaten noch ein separater Testdatensatz benötigt wird. 
* Erklären Sie, wie ein Feature-Vektor des ``CountVectorizer`` aufgebaut ist.

Sollten Sie sich unsicher sein, was die Korrektheit Ihrer Lösungen betrifft, fragen Sie während der Praktikumsstunde bitte nach! 

---

**Planung:**
1. Daten Laden & Splitten
4. Pipeline aufbauen
5. Trainieren + Predicten
6. Evaluieren und verbessern

---
**Importe**

In [1]:
import random
import os

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

# Spezielles
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

---
**Daten Laden & Splitten**

In [2]:
def load_txt(filename):
    with open(filename, "r") as file:
        text = file.read()
    return text

In [3]:
train_percentage_amount = 0.6
test_percentage_amount = 1.0
    
train_X = []
train_y = []
test_X = []
test_y = []
val_X = []
val_y = []

for category in ["business", "sport", "entertainment", "politics", "tech"]:
    datapath = "./data/data/bbc/"+category
    files = os.listdir(datapath)
    length = len(files)

    amount_traindata = int(length*train_percentage_amount)
    print(f"Amount Traindata: {amount_traindata}")
    amount_testdata = int(length*((1-train_percentage_amount)*test_percentage_amount))
    print(f"Amount Testdata: {amount_testdata}")
    amount_valdata = int(length*((1-train_percentage_amount)*(1-test_percentage_amount)))
    print(f"Amount Validationdata: {amount_valdata}")

    for i in range(amount_traindata+amount_testdata+amount_valdata): 
        if len(files) <= 0:
            break
        t = random.randrange(len(files))
        cur_elem = files[t]
        files.pop(t)

        if i >= 0 and i < amount_traindata:
            text = load_txt(datapath+"/"+cur_elem)
            train_X += [text]
            train_y += [category]
        elif i >= amount_traindata and i < amount_testdata:
            text = load_txt(datapath+"/"+cur_elem)
            test_X += [text]
            test_y += [category]
        else:
            text = load_txt(datapath+"/"+cur_elem)
            val_X += [text]
            val_y += [category]

Amount Traindata: 306
Amount Testdata: 204
Amount Validationdata: 0
Amount Traindata: 306
Amount Testdata: 204
Amount Validationdata: 0
Amount Traindata: 231
Amount Testdata: 154
Amount Validationdata: 0
Amount Traindata: 250
Amount Testdata: 166
Amount Validationdata: 0
Amount Traindata: 240
Amount Testdata: 160
Amount Validationdata: 0


In [4]:
print(len(train_X))
print(len(test_X))
print(len(val_X))

1333
0
888


---

**Pipeline aufbau**

In [5]:
count_vectorizer = CountVectorizer()
#clf = MultinomialNB()
clf = SVC()

pipeline = Pipeline([
    ("vectorizer", count_vectorizer),
    ("classifier", clf)
])

In [6]:
pipeline = pipeline.fit(train_X, train_y)

---

**1. Evaluierung**

In [7]:
cv_res = cross_validate(pipeline, train_X, train_y, cv=3)
cv_res

{'fit_time': array([1.41568542, 1.34254479, 1.33688688]),
 'score_time': array([0.58463407, 0.56164074, 0.53648043]),
 'test_score': array([0.90786517, 0.89414414, 0.89189189])}

In [8]:
cv_res['test_score'].mean()

0.8979670681917873

---

**Hyperparameter Anpassung**

GridSearch kann [in der Pipeline](https://stackoverflow.com/questions/43366561/use-sklearns-gridsearchcv-with-a-pipeline-preprocessing-just-once) sein oder [die Pipeline in GridSearch](https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html) (oder [das hier](https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html)).

CountVector GridSearchen?

Besser ohne Pipeline?

In [13]:
params = {'C':[0.001, 0.1, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 2.0],
         'kernel':['linear', 'poly', 'rbf', 'sigmoid']}
grid = GridSearchCV(pipeline, params)

In [14]:
grid.fit(train_X, train_y)

ValueError: Invalid parameter C for estimator Pipeline(steps=[('vectorizer', CountVectorizer()), ('classifier', SVC())]). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
grid_2.predict(test_X, test_y)

---

**Evaluierung**

---